In [1]:
import tejapi 
tejapi.ApiConfig.api_base="http://10.10.10.66"
tejapi.ApiConfig.api_key = "3jUCETU2KiPwGJeyETYOQd1TCoDoxX"
tejapi.ApiConfig.ignoretz = True
from TejToolAPI import *

## tejapi.get() vs TejToolAPI
## 情境一:
獲取財務資料(會計師簽證+自結+董事會決議數)
- 會計師簽證: TWN/AINVFINQ
    - 財報發布日: TWN/AINVFINQA
- 自結和董事會決議數: TWN/AFESTM1


### tejapi.get()

In [2]:
ticker = ['2330']
columns = ['r404','r401']
fin_auditor = tejapi.get('TWN/AINVFINQ',
                         coid = ticker,
                         opts = {'pivot':True}
                        )
fin_rep_ann_date = tejapi.get('TWN/AINVFINQA',
                         coid = ticker
                        )
# 欄位轉成英文小寫
fin_auditor.columns = fin_auditor.columns.str.lower()
fin_auditor.head(5)

,coid,mdate,key3,bp11,bp12,bp21,bp22,bp31,bp41,bp51,...,r609,r614,r611,r615,r307,r304,r305,r306,r316,r834
None,,,,,,,,,,,,,,,,,,,,,
0,2330,2005-03-01,A,60218990.0,0.0,22428991.0,0.0,13428985.0,8207745.0,229853709.0,...,NaN,21.96,35.94,None,NaN,NaN,NaN,NaN,NaN,2984.41
1,2330,2005-03-01,Q,60218990.0,0.0,22428991.0,0.0,13428985.0,8207745.0,229853709.0,...,NaN,21.96,35.94,None,NaN,NaN,NaN,NaN,NaN,2984.41
2,2330,2005-03-01,TTM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
3,2330,2005-06-01,A,88405154.0,0.0,32095014.0,0.0,15158776.0,7605043.0,257006435.0,...,NaN,19.52,38.91,None,NaN,NaN,NaN,NaN,NaN,5565.99
4,2330,2005-06-01,Q,88405154.0,0.0,32095014.0,0.0,15158776.0,7605043.0,257006435.0,...,NaN,18.76,37.38,None,NaN,NaN,NaN,NaN,NaN,2914.70


### 相同發布日下，取財務年月最大的資料

In [3]:
fin_rep_ann_date.groupby(['coid','annd']).count()

mdate  key3  no  sem  merg  curr  fin_ind
coid annd                                                 
2330 2006-05-02      6     6   6    6     6     6        6
     2006-07-28      6     6   6    6     6     6        6
     2006-10-30      3     3   3    3     3     3        3
     2007-02-26      4     4   4    4     4     4        4
     2007-07-26      6     6   6    6     6     6        6
...                ...   ...  ..  ...   ...   ...      ...
     2022-02-25      6     6   6    6     6     6        6
     2022-05-13      6     6   6    6     6     6        6
     2022-08-12      6     6   6    6     6     6        6
     2022-11-14      6     6   6    6     6     6        6
     2023-02-24      6     6   6    6     6     6        6

[63 rows x 7 columns]

In [4]:
fin_rep_ann_date.query('annd == "2006-07-28"')

,coid,mdate,key3,no,sem,merg,curr,annd,fin_ind
None,,,,,,,,,
3,2330,2005-06-01,A,001,2,Y,TWN,2006-07-28,F
4,2330,2005-06-01,Q,001,2,Y,TWN,2006-07-28,F
5,2330,2005-06-01,TTM,001,2,Y,TWN,2006-07-28,F
15,2330,2006-06-01,A,001,2,Y,TWN,2006-07-28,F
16,2330,2006-06-01,Q,001,2,Y,TWN,2006-07-28,F
17,2330,2006-06-01,TTM,001,2,Y,TWN,2006-07-28,F


In [5]:
# 排序
fin_rep_ann_dates = fin_rep_ann_date.sort_values(['coid','annd','mdate'])
fin_rep_ann_dates = fin_rep_ann_dates.drop_duplicates(subset = ['coid','annd','key3'], keep = 'last').reset_index(drop=True)
fin_rep_ann_dates

,coid,mdate,key3,no,sem,merg,curr,annd,fin_ind
0,2330,2006-03-01,A,001,1,N,TWN,2006-05-02,F
1,2330,2006-03-01,Q,001,1,N,TWN,2006-05-02,F
2,2330,2006-03-01,TTM,001,1,N,TWN,2006-05-02,F
3,2330,2006-06-01,A,001,2,Y,TWN,2006-07-28,F
4,2330,2006-06-01,Q,001,2,Y,TWN,2006-07-28,F
...,...,...,...,...,...,...,...,...,...
182,2330,2022-09-01,Q,001,3,Y,TWN,2022-11-14,F
183,2330,2022-09-01,TTM,001,3,Y,TWN,2022-11-14,F
184,2330,2022-12-01,A,001,4,Y,TWN,2023-02-24,F
185,2330,2022-12-01,Q,001,4,Y,TWN,2023-02-24,F


In [6]:
fin_rep_ann_dates.query('annd == "2006-07-28"')

,coid,mdate,key3,no,sem,merg,curr,annd,fin_ind
3,2330,2006-06-01,A,001,2,Y,TWN,2006-07-28,F
4,2330,2006-06-01,Q,001,2,Y,TWN,2006-07-28,F
5,2330,2006-06-01,TTM,001,2,Y,TWN,2006-07-28,F


### 合併發布日與財報資料

In [7]:
fin_auditor_new = fin_rep_ann_date.merge(fin_auditor, on = ['coid', 'mdate', 'key3'])
fin_auditor_new = fin_auditor_new.loc[:,['coid', 'mdate', 'key3', 'annd']+columns]
fin_auditor_new

,coid,mdate,key3,annd,r404,r401
0,2330,2005-03-01,A,2006-05-02,0.00,0.00
1,2330,2005-03-01,Q,2006-05-02,NaN,NaN
2,2330,2005-03-01,TTM,2006-05-02,NaN,NaN
3,2330,2005-06-01,A,2006-07-28,0.00,0.00
4,2330,2005-06-01,Q,2006-07-28,NaN,NaN
...,...,...,...,...,...,...
553,2330,2022-09-01,Q,2022-11-14,82.16,47.86
554,2330,2022-09-01,TTM,2022-11-14,NaN,NaN
555,2330,2022-12-01,A,2023-02-24,72.54,42.61
556,2330,2022-12-01,Q,2023-02-24,81.08,42.75


### 取得自結資料

In [8]:
fin_self_acc = tejapi.get('TWN/AFESTM1',
                         coid = ticker,
                          opts = {'columns':['coid', 'mdate', 'key3','annd']+columns}
                        )
fin_self_acc.head(5)

,coid,mdate,key3,annd,r404,r401
None,,,,,,
0,2330,2006-09-01,A,2006-10-26,NaN,NaN
1,2330,2006-09-01,Q,2006-10-26,NaN,NaN
2,2330,2006-12-01,A,2007-01-25,41.48,19.07
3,2330,2006-12-01,TTM,2007-01-25,41.48,19.07
4,2330,2007-03-01,A,2007-07-26,NaN,NaN


In [9]:
fin_self_acc

,coid,mdate,key3,annd,r404,r401
None,,,,,,
0,2330,2006-09-01,A,2006-10-26,NaN,NaN
1,2330,2006-09-01,Q,2006-10-26,NaN,NaN
2,2330,2006-12-01,A,2007-01-25,41.48,19.07
3,2330,2006-12-01,TTM,2007-01-25,41.48,19.07
4,2330,2007-03-01,A,2007-07-26,NaN,NaN
...,...,...,...,...,...,...
355,2330,2022-12-01,A,2023-02-14,72.54,42.61
356,2330,2022-12-01,Q,2023-01-12,81.08,42.75
357,2330,2022-12-01,Q,2023-02-14,81.08,42.75


In [10]:
fin_data = pd.concat([fin_self_acc, fin_auditor_new]).sort_values(['coid','mdate','annd']).reset_index(drop=True)
fin_data = fin_data.drop_duplicates(subset = ['coid', 'mdate','key3'], keep = 'last').reset_index(drop=True)
fin_data

,coid,mdate,key3,annd,r404,r401
0,2330,2005-03-01,A,2006-05-02,0.00,0.00
1,2330,2005-03-01,Q,2006-05-02,NaN,NaN
2,2330,2005-03-01,TTM,2006-05-02,NaN,NaN
3,2330,2005-06-01,A,2006-07-28,0.00,0.00
4,2330,2005-06-01,Q,2006-07-28,NaN,NaN
...,...,...,...,...,...,...
209,2330,2022-09-01,Q,2022-11-14,82.16,47.86
210,2330,2022-09-01,TTM,2022-11-14,NaN,NaN
211,2330,2022-12-01,A,2023-02-24,72.54,42.61
212,2330,2022-12-01,Q,2023-02-24,81.08,42.75


### 降為日頻

In [11]:
days = pd.date_range(start='2006-05-01', end = '2022-12-31', freq='d')
days = pd.DataFrame({'all_dates':days})
days

,all_dates
0,2006-05-01
1,2006-05-02
2,2006-05-03
3,2006-05-04
4,2006-05-05
...,...
6084,2022-12-27
6085,2022-12-28
6086,2022-12-29
6087,2022-12-30


In [12]:
fin_data_new = days.merge(fin_data, left_on = 'all_dates', right_on = 'annd', how = 'left').ffill()
fin_data_new

,all_dates,coid,mdate,key3,annd,r404,r401
0,2006-05-01,NaN,NaT,NaN,NaT,NaN,NaN
1,2006-05-02,2330,2005-03-01,A,2006-05-02,0.00,0.00
2,2006-05-02,2330,2005-03-01,Q,2006-05-02,0.00,0.00
3,2006-05-02,2330,2005-03-01,TTM,2006-05-02,0.00,0.00
4,2006-05-02,2330,2006-03-01,A,2006-05-02,113.06,39.89
...,...,...,...,...,...,...,...
6231,2022-12-27,2330,2022-09-01,TTM,2022-11-14,82.16,47.86
6232,2022-12-28,2330,2022-09-01,TTM,2022-11-14,82.16,47.86
6233,2022-12-29,2330,2022-09-01,TTM,2022-11-14,82.16,47.86
6234,2022-12-30,2330,2022-09-01,TTM,2022-11-14,82.16,47.86


In [20]:
fin_data_new.query('all_dates > "2006-12-31" and all_dates < "2007-03-01"')

,all_dates,coid,mdate,key3,annd,r404,r401
257,2007-01-01,2330,2005-09-01,TTM,2006-10-30,0.00,0.00
258,2007-01-02,2330,2005-09-01,TTM,2006-10-30,0.00,0.00
259,2007-01-03,2330,2005-09-01,TTM,2006-10-30,0.00,0.00
260,2007-01-04,2330,2005-09-01,TTM,2006-10-30,0.00,0.00
261,2007-01-05,2330,2005-09-01,TTM,2006-10-30,0.00,0.00
...,...,...,...,...,...,...,...
314,2007-02-26,2330,2005-12-01,Q,2007-02-26,0.00,0.00
315,2007-02-26,2330,2005-12-01,TTM,2007-02-26,0.00,0.00
316,2007-02-26,2330,2006-12-01,A,2007-02-26,41.48,18.53
317,2007-02-27,2330,2006-12-01,A,2007-02-26,41.48,18.53


In [17]:
fin_self_acc.query('annd > "2006-12-01" and annd < "2007-12-31"')

,coid,mdate,key3,annd,r404,r401
None,,,,,,
2,2330,2006-12-01,A,2007-01-25,41.48,19.07
3,2330,2006-12-01,TTM,2007-01-25,41.48,19.07
4,2330,2007-03-01,A,2007-07-26,NaN,NaN
5,2330,2007-03-01,Q,2007-07-26,NaN,NaN
6,2330,2007-06-01,A,2007-07-27,-30.49,-12.60
7,2330,2007-06-01,TTM,2007-07-27,-13.14,-4.02
8,2330,2007-09-01,A,2007-10-25,-20.74,-5.64
9,2330,2007-09-01,Q,2007-10-25,-1.29,7.86


In [18]:
fin_auditor_new.query('annd > "2006-12-01" and annd < "2007-12-31"')

,coid,mdate,key3,annd,r404,r401
9,2330,2005-12-01,A,2007-02-26,0.00,0.00
10,2330,2005-12-01,Q,2007-02-26,NaN,NaN
11,2330,2005-12-01,TTM,2007-02-26,NaN,NaN
18,2330,2006-09-01,A,2007-10-30,77.78,32.42
19,2330,2006-09-01,Q,2007-10-30,43.18,22.14
20,2330,2006-09-01,TTM,2007-10-30,NaN,NaN
21,2330,2006-12-01,A,2007-02-26,41.48,18.53
22,2330,2007-03-01,A,2007-07-26,-42.09,-17.00
23,2330,2007-03-01,Q,2007-07-26,-42.09,-17.00
24,2330,2007-03-01,TTM,2007-07-26,NaN,NaN
